In [1]:
%load_ext autotime

In [2]:
import boto3
import datetime
import logging
import json
import pickle
import pytest
import sys
import time

from etltools import s3

reload(logging)  # get around notebook problem

<module 'logging' from '/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/logging/__init__.pyc'>

time: 523 ms


In [3]:
logging.basicConfig(
    level=logging.INFO, 
    format='[%(asctime)s] {%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
#     handlers=[
#         logging.FileHandler(filename='mylog.log', mode='w'),
#         logging.StreamHandler(sys.stdout),
#     ]
)

time: 1.42 ms


In [4]:
# Test whether logging works:
logger = logging.getLogger()
logger.info('TEST INFO')

[2018-01-08 23:10:14,544] {<ipython-input-4-efd2929c73d3>:3} INFO - TEST INFO


time: 1.81 ms


In [5]:
aws_info = {
    'profile_name': 'sandbox',
}

uids = s3.read_json('s3://picwell.sandbox.medicare/samples/philadelphia-2015-1k-sample')
pids = ['2820028008119', '2820088001036']

time: 274 ms


# Test ConfigInfo

In [6]:
from lambda_client.config_info import ConfigInfo

configs = ConfigInfo('lambda_client/lambda.cfg')

print configs.claims_bucket
print configs.claims_path
print
print configs.benefits_bucket
print configs.benefits_path
print
print configs.claims_table

picwell.sandbox.analytics
junghoon/lambda_calculator

picwell.sandbox.analytics
junghoon/lambda_calculator_benefits

ma_claims
time: 5.51 ms


In [7]:
all_states = configs.all_states

print '{} states'.format(len(all_states))
print all_states

51 states
['01', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '72']
time: 1.03 ms


# Test ClaimsClient

In [8]:
from lambda_client import ClaimsClient

time: 561 µs


In [9]:
# Test S3:
client = ClaimsClient(aws_info, 
                      s3_bucket=configs.claims_bucket,
                      s3_path=configs.claims_path)

people = client.get(uids[:1])
print 'claims of {} people retrieved'.format(len(people))

claims of 1 people retrieved
time: 259 ms


In [10]:
person = people[0]
print person.keys()
{
    'uid': person['uid'],
    'medical_claims': person['medical_claims'][:5]
}

[u'medical_claims', u'uid']


{'medical_claims': [{u'admitted': u'2014-04-03',
   u'benefit_category': 16,
   u'cost': u'148.0',
   u'discharged': u'2014-04-03',
   u'length_of_stay': 1},
  {u'admitted': u'2014-05-02',
   u'benefit_category': 16,
   u'cost': u'74.55',
   u'discharged': u'2014-05-02',
   u'length_of_stay': 1},
  {u'admitted': u'2014-05-05',
   u'benefit_category': 16,
   u'cost': u'104.39',
   u'discharged': u'2014-05-05',
   u'length_of_stay': 1},
  {u'admitted': u'2014-05-13',
   u'benefit_category': 16,
   u'cost': u'210.12',
   u'discharged': u'2014-05-13',
   u'length_of_stay': 1},
  {u'admitted': u'2014-05-19',
   u'benefit_category': 11,
   u'cost': u'442.2',
   u'discharged': u'2014-05-19',
   u'length_of_stay': 1}],
 'uid': u'1302895801'}

time: 5.02 ms


In [11]:
# Test DynamoDB:
client = ClaimsClient(aws_info,
                      table_name=configs.claims_table)

people = client.get(uids[:1])
print 'claims of {} people retrieved'.format(len(people))

claims of 1 people retrieved
time: 248 ms


In [12]:
# Test configuration file and retrieving multiple people:
client = ClaimsClient(aws_info)

people = client.get(uids[:5])
print 'claims of {} people retrieved'.format(len(people))

claims of 5 people retrieved
time: 717 ms


In [13]:
# The object should not be pickled.
with pytest.raises(Exception, match='ClaimsClient object cannot be pickled.'):
    pickle.dumps(client)

time: 1.27 ms


# Test BenefitsClient

In [14]:
from lambda_client import BenefitsClient

time: 720 µs


In [15]:
client = BenefitsClient(aws_info)

print client.all_states

['01', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '72']
time: 1.83 ms


In [16]:
plans = client._get_one_state('01')
print '{} plans read for state 01'.format(len(plans))

plans = client._get_one_state('04')
print '{} plans read for state 04'.format(len(plans))

47 plans read for state 01
75 plans read for state 04
time: 903 ms


In [17]:
plans = client.get_by_state(['01', '04'])
print '{} plans read'.format(len(plans))

122 plans read
time: 593 ms


In [18]:
plans = client.get_all()
print '{} plans read'.format(len(plans))

3558 plans read
time: 9.45 s


In [19]:
# Compare the timing against reading the entire file:
from lambda_client.storage_utils import _read_json

session = boto3.Session(**aws_info)
resource = session.resource('s3')

time: 67.1 ms


In [20]:
all_plans = _read_json('picwell.sandbox.medicare', 'ma_benefits/cms_2018_pbps_20171005.json', resource)

print '{} plans read'.format(len(plans))

3558 plans read
time: 4.25 s


In [21]:
# Ensure that the same plans are read:
sort_key = lambda plan: plan['picwell_id']
assert sorted(all_plans, key=sort_key) == sorted(plans, key=sort_key)

time: 151 ms


In [22]:
# The object should not be pickled.
with pytest.raises(Exception, match='BenefitsClient object cannot be pickled.'):
    pickle.dumps(client)

time: 1.83 ms


# Test Cost Breakdown

In [23]:
from lambda_client import CalculatorClient

client = CalculatorClient(aws_info)

time: 952 µs


In [24]:
responses = client._get_one_breakdown(uids[0], pids, '01')

print '{} responses returned'.format(len(responses))
responses[0]

2 responses returned


{u'allowed': 26376.640000000003,
 u'covered_breakdown': {u'categories': {u'0': 0.0,
   u'11': 186.33999999999997,
   u'13': 13.23,
   u'15': 30.79,
   u'16': 0.0,
   u'19': 80.0,
   u'25': 1750.0,
   u'30': 719.1700000000001,
   u'31': 300.0,
   u'44': 0.0,
   u'49': 118.48800000000001,
   u'7': 479.558},
  u'composite': 3677.5759999999996,
  u'in_network': 3677.5759999999996,
  u'out_network': 0.0},
 u'deductible_breakdown': {u'categories': {u'0': 0.0,
   u'11': 0.0,
   u'13': 0.0,
   u'15': 0.0,
   u'16': 0.0,
   u'19': 0.0,
   u'25': 0.0,
   u'30': 0.0,
   u'31': 0.0,
   u'44': 0.0,
   u'49': 0.0,
   u'7': 0.0},
  u'composite': 0.0,
  u'in_network': 0.0,
  u'out_network': 0.0},
 u'oop': 3677.5759999999996,
 u'picwell_id': u'2820028008119',
 u'uid': u'1302895801',
 u'uncovered': 0.0,
 u'uncovered_breakdown': {u'categories': {u'0': 0.0,
   u'11': 0.0,
   u'13': 0.0,
   u'15': 0.0,
   u'16': 0.0,
   u'19': 0.0,
   u'25': 0.0,
   u'30': 0.0,
   u'31': 0.0,
   u'44': 0.0,
   u'49': 0.0,


time: 3.91 s


In [25]:
responses = client.get_breakdown(uids[:1], pids)

print '{} responses returned'.format(len(responses))

2 responses returned
time: 3.58 s


In [27]:
# It seems like
#
# (a) the time it takes to issue threads increases linearly; and
# (b) the time it takes for the last thread to finish is about 5 seconds all the time. 
for num_people in range(2, 15):
    start = datetime.datetime.now()
    responses = client.get_breakdown(uids[:num_people], pids)
    elapsed = (datetime.datetime.now() - start).total_seconds()

    print '{} responses returned ({} seconds)'.format(len(responses), elapsed)

4 responses returned (4.475258 seconds)


KeyError: 'StatusCode'

time: 14.1 s


In [28]:
# Let's try something larger:
responses = client.get_breakdown(uids, pids)

print '{} responses returned'.format(len(responses))

KeyError: 'StatusCode'

time: 6.37 s


In [29]:
# Run calculcations locally for comparison:
from lambda_package.calc.calculator import calculate_oop

claims_client = ClaimsClient(aws_info)
people = claims_client.get(uids)

benefits_client = BenefitsClient(aws_info)
plans = benefits_client.get_by_pid(pids)

costs = []
for person in people:
    claims = person['medical_claims']
    
    for plan in plans:
        cost = calculate_oop(claims, plan)
        cost.update({
            'uid': person['uid'],
            'picwell_id': str(plan['picwell_id']),
        })
        
        costs.append(cost)
        
print '{} costs calculated'.format(len(costs))

2000 costs calculated
time: 2min 50s


# Test Batch Calculation

In [ ]:
# uids = s3.read_json('s3n://picwell.sandbox.medicare/samples/philadelphia-2015')

# print '{} uids read'.format(len(uids))

In [ ]:
# uids[:10]

In [ ]:
# requests_per_second = 100

# for uid in uids:
# #     client.calculate_async(uid, months=['01'])
#     client.calculate_async(uid)
#     time.sleep(1.0/requests_per_second)  